Instalando Pacotes

In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install pandas

Fazendo a Raspagem dos dados

In [43]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

url = "https://www.imdb.com/"

# Precisamos desse Headers para que o site aceita a solicitação
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

# Solicitando para obter o conteúdo da página
response = requests.get(url, headers=headers)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

top250 = soup.find('ul', class_='ipc-list navlinkcat__list ipc-list--baseAlt')
top250 = top250.find_all('a')[1]
top250 = "https://www.imdb.com/" + top250.get('href')

# Agora podemos acessar o link que geramos acima da mesma forma que acessamos o outro:
response = requests.get(top250, headers=headers)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

# Quero selecionar todos os nomes dos filmes
top250_movies = [h3.text for h3 in soup.select('ul h3')]


df = pd.DataFrame({'Nome do Filme': top250_movies})
print(df)

                          Nome do Filme
0              1. Um Sonho de Liberdade
1                  2. O Poderoso Chefão
2    3. Batman - O Cavaleiro das Trevas
3               4. O Poderoso Chefão II
4           5. 12 Homens e uma Sentença
..                                  ...
245             246. Os Incompreendidos
246     247. Quando Duas Mulheres Pecam
247                248. A Vida de Brian
248                        249. Aladdin
249                250. Dança com Lobos

[250 rows x 1 columns]
